# This notebook demonstrates computing and displayihng Dose Volume Histograms from segmentation and dose distribution stored in planC

In [1]:
import oct2py

In [2]:
# Without using Octave magic
#oc = oct2py.Oct2Py(temp_dir=oc_temp_dir)
#oc.version()

## Define directory to store temporary variables for I/O between Octave and Python
### Select a location that has fast disk read/write access

In [3]:
oc_temp_dir = "C:/software/octave_temp_dir"

In [4]:
import os
if not os.path.isdir(oc_temp_dir):
    os.mkdir(oc_temp_dir)

## Load Octave cell magic

In [5]:
%load_ext oct2py.ipython

## Add CERR to Octave path

In [6]:
%%octave -t {oc_temp_dir}
cerrPath = 'C:/software/CERR_octave_dev/CERR'
addpath(genpath(cerrPath));
warning('off')
eval('pkg load io');
eval('pkg load image');
eval('pkg load statistics');

cerrPath = C:/software/CERR_octave_dev/CERR

## Compute DVH and make it available to Python

In [7]:
%%octave -t {oc_temp_dir}
sampleData = fullfile(getCERRPath(),'..','Unit_Testing','data_for_cerr_tests', ...
              'CERR_plans','head_neck_ex1_20may03.mat.bz2');
doseNum = 1;
binWidth = 0.01;

planC = loadPlanC(sampleData, tempdir());
planC = updatePlanFields(planC);
planC = quality_assure_planC(sampleData, planC);
indexS = planC{end};

% Get Structure names from planC or define your own to extract DVH
indexS = planC{end};
strC = {planC{indexS.structures}.structureName};

doses = {length(strC)};
vols = {length(strC)};

for i = 1:length(strC)
    structNum = getMatchingIndex(strC(i), strC, 'exact');
    scanNum = getStructureAssociatedScan(structNum ,planC);        
    [doseV,volV] = getDVH(structNum,doseNum,planC);
    [doseBinsV, volHistV] = doseHist(doseV, volV, binWidth);
    doses{i} = doseBinsV;
    vols{i} = volHistV;
end

CERR>>  Decompressing head_neck_ex1_20may03.mat.bz2...

7-Zip 4.65  Copyright (c) 1999-2009 Igor Pavlov  2009-02-03

Processing archive: C:\software\CERR_octave_dev\CERR\CERR_core\..\Unit_Testing\data_for_cerr_tests\CERR_plans\head_neck_ex1_20may03.mat.bz2

Extracting  head_neck_ex1_20may03.mat

Everything is Ok

Size:       68680824

Compressed: 6748538

CERR>>  Loading head_neck_ex1_20may03.mat.bz2...

CERR>>  Loaded head_neck_ex1_20may03.mat.bz2...

Pull 'doses', 'vols' and 'strC' variables into Python (They are currently Octave variables)

In [8]:
%octave_pull doses vols strC

## Plot DVH

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import HBox, VBox

dose_list = doses[0].tolist()
vols_list = vols[0].tolist()

#Convert absolute volumes to relative volumes (i.e. percentage)
def toRelativeVolV(volHistV):
    cumVolsV = np.cumsum(volHistV)
    cumVols2V = cumVolsV[-1] - cumVolsV
    relativeVolV = cumVols2V / cumVolsV[-1]
    return relativeVolV

def update_checkbox(change):
    with output:
        update_graph(change['new'])

boxes = []

for i in range(len(dose_list)):
    box = widgets.Checkbox(
        value=True,
        description=strC[0][i],
        disabled=False,
        indent=False,
    )
    boxes.append(box)
    boxes[i].observe(update_checkbox, names='value')

#Arrange checkboxes in grid
left_col = []    
middle_col = []
right_col = []

for i in range(len(boxes)):
    if(i < len(boxes) * .34):
        left_col.append(boxes[i])
    elif(i < len(boxes) * .67):
        middle_col.append(boxes[i])
    else:
        right_col.append(boxes[i])

def update_graph(x):
    clear_output(wait=True)
       
    fig, ax = plt.subplots()
    
    for i in range(len(dose_list)):
        if(boxes[i].value):
            ax.plot(dose_list[i][0], toRelativeVolV(vols_list[i]) * 100, label=strC[0][i])
    
    ax.legend(loc="upper right")
    
    display(widgets.Output())
    plt.ylabel('Volume (%)')
    plt.xlabel('Dose (Gy)')
    plt.rcParams["figure.figsize"] = (13, 7)
    plt.show()
          
output = widgets.Output()
display(output)

display(HBox([VBox(left_col), VBox(middle_col), VBox(right_col)]))

#Uncheck and recheck the first box to display the graph
boxes[0].value = False
boxes[0].value = True

Output()